## 3. C++ Python extensions
Using the GridTools backends we can also create language bindings that allow to call the C++ / CUDA implemented stencil from python.
This is very similar to the default behaviour when creating and using a stencil directly in `gt4py` code.



In [1]:
!gtpyc --help

Usage: gtpyc [OPTIONS] INPUT_PATH

  GT4Py (GridTools for Python) stencil compiler.

  INPUT_PATH can be either a gtscript or python file or a python package.

Options:
  -b, --backend [debug|numpy|gtx86|gtmc|gtcuda]
                                  Choose a backend
  -o, --output-path DIRECTORY     output path for the compiled source.
  --bindings BINDINGS             Generate source code for language bindings.
                                  Which languages are available depends on the
                                  backend.

  -C, --compile-bindings          Compile bindings source code. Only has an
                                  effect in combination wih --bindings.

  -O, --option OTPION             Backend option (multiple allowed), format:
                                  -O key=value

  -E, --externals JSON            JSON string describing externals overrides
                                  for all stencils.

  -s, --silent                    suppress output.
  --h

In [2]:
!gtpyc -b "gtx86" --bindings=python --compile-bindings ./stencils.gt.py

reading input file stencils.gt.py
input file loaded as module <module 'stencils' from '/Users/ricoh/Code/gt4py-tutorial/gtpyc tutorial/examples/simple-stencils/stencils.gt.py'>
Found 2 stencils: "axpy", "dot".
Building stencil axpy
 -> /Users/ricoh/Code/gt4py-tutorial/gtpyc tutorial/examples/simple-stencils/src_axpy/computation.hpp
 -> /Users/ricoh/Code/gt4py-tutorial/gtpyc tutorial/examples/simple-stencils/src_axpy/computation.cpp
 -> /Users/ricoh/Code/gt4py-tutorial/gtpyc tutorial/examples/simple-stencils/src_axpy/bindings.cpp
compiled python bindings to /Users/ricoh/Code/gt4py-tutorial/gtpyc tutorial/examples/simple-stencils/_axpy.cpython-37m-darwin.so
 -> /Users/ricoh/Code/gt4py-tutorial/gtpyc tutorial/examples/simple-stencils/axpy.py
Building stencil dot
 -> /Users/ricoh/Code/gt4py-tutorial/gtpyc tutorial/examples/simple-stencils/src_dot/computation.hpp
 -> /Users/ricoh/Code/gt4py-tutorial/gtpyc tutorial/examples/simple-stencils/src_dot/computation.cpp
 -> /Users/ricoh/Code/gt4py-

In [3]:
!ls

1-gtscript-to-python.ipynb  axpy.py
2-gtscript-to-c++.ipynb     dot.py
3-c++-python-bindings.ipynb src_axpy
_axpy.so                    src_dot
_dot.so                     stencils.gt.py


In [4]:
import numpy
from axpy import axpy as axpy_cls
axpy_f = axpy_cls()

In [5]:
x = numpy.array([[[1, 2], [1, 4]]], dtype=numpy.float64)
y = numpy.array([[[2, 3], [4, 4]]], dtype=numpy.float64)

As you will see, it is not yet possible to call stencils compiled with the `gt<x>` family of backends with numpy arrays.

In [6]:
try:
    axpy_f(x, y, origin=(0, 0, 0))
except AttributeError:
    print("Calling with numpy array failed!")

Calling with numpy array failed!


However, this is being worked on. In the meantime we can still call the stencil with storages we can import from `gt4py`.

In [7]:
from gt4py import storage

ORIG = (0, 0, 0)
x = storage.from_array(x, backend=axpy_f.backend, default_origin=ORIG)
y = storage.from_array(y, backend=axpy_f.backend, default_origin=ORIG)

axpy_f(x, y)
y

CPUStorage([[[3., 5.],
             [5., 8.]]])

### Cleanup
Disable the following cell to keep generated files around. Run manually to recreate the initial state.

In [8]:
import pathlib
import shutil
cwd = pathlib.Path('.')
axpy_mod = cwd / 'axpy.py'
dot_mod = cwd / 'dot.py'
axpy_ext = cwd / '_axpy.so'
dot_ext = cwd / '_dot.so'
pycache = cwd / '__pycache__'
axpy_src = cwd / 'src_axpy'
dot_src = cwd / 'src_dot'
files = [axpy_mod, dot_mod, axpy_ext, dot_ext]
dirs = [pycache, axpy_src, dot_src]

for file in files:
    if file.exists():
        file.unlink()
for folder in dirs:   
    if folder.exists():
        shutil.rmtree(folder)